Two funcs for checking the .tfrec<p>
    To see if there's any missing or mistake in the saved data in .tfrec.

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import IPython.display as display

import xlrd
import numpy as np
import pandas as pd
import os
import re
from progressbar import *
import nibabel as nib
import pdb
import scipy.ndimage
import matplotlib.pyplot as plt
from my_tools import *
import tarfile
import shutil

In [ ]:
'''
This could be used to check if the generated .tfrec file has the right data
This is using tf.python_io.tf_record_iterator(path='try_training.tfrec')
The next one is using tf.data.TFRecordDataset('try_training.tfrec')
'''

record_iterator = tf.python_io.tf_record_iterator(path='try_training.tfrec')

SHAPE = np.load('./data_npy/mean_npy.npy').shape

info_df = pd.read_csv('./training.csv', sep=',',header=0)

# pbar = ProgressBar().start()
for i,string_record in enumerate(record_iterator):
    
    
    example = tf.train.Example()
    example.ParseFromString(string_record)

    sub_id = example.features.feature['id'].int64_list.value[0]
    
    decode_label = example.features.feature['label'].float_list.value
    decode_label = np.round(decode_label,2)[0]
#     print(sub_id,decode_label)
    decode_arr = example.features.feature['arr_raw'].bytes_list.value
    decode_arr = tf.decode_raw(decode_arr,tf.float32)
    decode_arr = tf.reshape(decode_arr,SHAPE)
    decode_arr = decode_arr.numpy()
    
    npy_filename = str(sub_id) + '.npy'
    npy_path_filename = os.path.join(npy_dir,npy_filename)
    
    try:
        arr_npy = np.load(npy_path_filename)
    except FileNotFoundError:
        print('No such file: ',npy_filename)
        continue
    label = round(info_df['age'][info_df['id']==sub_id].values[0],2)
    arr = arr_npy.astype(np.float32)
#     break
    
    if label != decode_label or np.sum(decode_arr!=arr)!=0:
        print_sep()
        print(i)
        print(npy_path_filename)
        print_sep()
        print('original label: ',label)
#         print2d(arr)
        print_sep()
        print('extracted label: ',decode_label)
#         print2d(decode_arr)
        print('label == decode_label: ',label==decode_label)
        print('arr == decode_arr: ',np.sum(decode_arr!=arr))

In [ ]:
'''
This one is using tf.data.TFRecordDataset('try_training.tfrec')
'''

SHAPE = np.load('./data_npy/mean_npy.npy').shape
info_df = pd.read_csv('./training.csv', sep=',',header=0)

raw_image_dataset = tf.data.TFRecordDataset('try_training.tfrec')

# Create a dictionary describing the features.  
image_feature_description = {
    'arr_raw':tf.FixedLenFeature([],tf.string),
    'label': tf.FixedLenFeature([],tf.float32),
    'id': tf.FixedLenFeature([],tf.int64),
}

def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
# type(parsed_image_dataset)

for image_features in parsed_image_dataset:
    decode_arr = tf.decode_raw(image_features['arr_raw'],tf.float32)
    decode_arr = tf.reshape(decode_arr,SHAPE)
    decode_arr = decode_arr.numpy()
#     print(type(decode_arr))
#     print(decode_arr.shape)
    decode_label = image_features['label']
    decode_label = np.round(decode_label,2)
#     print(type(decode_label))
#     print(decode_label)
    decode_id = image_features['id']
    decode_id = int(decode_id)
#     print(type(decode_id))
#     print(decode_id)
#     break

    npy_filename = str(decode_id) + '.npy'
    npy_path_filename = os.path.join(npy_dir,npy_filename)
    
    try:
        arr_npy = np.load(npy_path_filename)
    except FileNotFoundError:
        print('No such file: ',npy_filename)
        continue
    label = round(info_df['age'][info_df['id']==decode_id].values[0],2)
    arr = arr_npy.astype(np.float32)
#     break
    
    if round(label-decode_label,2)!=0 or np.sum(decode_arr!=arr)!=0:
        print_sep()
        print(npy_path_filename)
        print_sep()
        print('original label: ',label)
        print2d(arr)
        print_sep()
        print('extracted label: ',decode_label)
        print2d(decode_arr)
        print('label == decode_label: ',label==decode_label)
        print('arr == decode_arr: ',np.sum(decode_arr!=arr))
#     break

In [ ]:
'''
In this cell, we save an ndarray whose .dtype is int64 into a .tfrec and read it out.
It proves that nothing changed.
BINGO!
'''

arr_npy = np.load('../age_demo_ABIDE/data_npy/mean_npy.npy')
arr_string = arr_npy.tostring()
with tf.python_io.TFRecordWriter('test.tfrec') as writer:
    example = tf.train.Example(
        features = tf.train.Features(
            feature = {
                'arr_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[arr_string])),
            }
        )
    )
    writer.write(example.SerializeToString())

SHAPE = np.load('../age_demo_ABIDE/data_npy/mean_npy.npy').shape

raw_image_dataset = tf.data.TFRecordDataset('test.tfrec')
image_feature_description = {
    'arr_raw':tf.FixedLenFeature([],tf.string),
}
def _parse_image_function(example_proto):
    # Parse the input tf.Example proto using the dictionary above.
    return tf.parse_single_example(example_proto, image_feature_description)
parsed_image_dataset = raw_image_dataset.map(_parse_image_function)

for image_features in parsed_image_dataset:
    decode_arr = tf.decode_raw(image_features['arr_raw'],tf.int64)
    decode_arr = tf.reshape(decode_arr,SHAPE)
    decode_arr = decode_arr.numpy()
    
print(arr_npy.dtype == decode_arr.dtype)
print(np.sum(arr_npy - decode_arr))